In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import nltk
nltk.download('stopwords')
import os
import string
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\omegh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def lowercase_data(text):
  lowercase=text.lower()
  #print(lowercase)
  return lowercase

In [5]:
#Tokenize
def tokenize_Data(text):
  tokenized_words= TweetTokenizer().tokenize(text)
  #print(tokenized_words)
  return tokenized_words

In [6]:
#Removing stopwords
def remove_stopwords(tokens):
  stop_words=set(stopwords.words('english'))
  words_without_stopwords=[word for word in tokens if word not in stop_words]
  #print(words_without_stopwords)
  return words_without_stopwords

In [7]:
#Removing punctuations
def remove_punctuations(tokens):
  removed_punct = [word for word in tokens if word not in string.punctuation]
  #print(removed_punct)
  return removed_punct

In [8]:
#Removing blank space
def remove_blank_space(tokens):
  removed_blankspace = [word for word in tokens if word.strip()]
 # print(removed_blankspace)
  return removed_blankspace

In [9]:
input_folder_path="text_files"
output_folder_path="Processed_files"
for file in os.listdir(input_folder_path):
  input_file_path=os.path.join(input_folder_path,file)
  output_file_path=os.path.join(output_folder_path,file)
    
  with open(input_file_path,'r',encoding='utf-8') as input_file:
    text=input_file.read()
    text=lowercase_data(text)
    tokens=tokenize_Data(text)
    tokens=remove_stopwords(tokens)
    tokens=remove_punctuations(tokens)
    tokens=remove_blank_space(tokens)

    processed_data=' '.join(tokens)
    with open(output_file_path,'w',encoding='utf-8') as output_file:
        output_file.write(processed_data)
print("complete")
      

complete


In [10]:
#Q2.Unigram Inverted Index and Boolean Queries
import pickle
def inverted_index_create(processes_data_folder):
  inverted_index={}
  for file in os.listdir(processes_data_folder):
    file_path=os.path.join(processes_data_folder,file)
    with open(file_path,'r',encoding='utf-8') as processed:
      processed_Data=processed.read()
      tokens=processed_Data.split()
      for token in tokens:
        if token not in inverted_index:
          inverted_index[token]=set()
        inverted_index[token].add(file)
  return inverted_index

inverted_index=inverted_index_create("Processed_files")


In [11]:
#2.Python pickle module
with open('inverted_index.pickle','wb') as f:
  pickle.dump(inverted_index,f,protocol=pickle.HIGHEST_PROTOCOL)
  #pickle data to a serialize format

with open('inverted_index.pickle','rb') as f:
  loaded_index=pickle.load(f)
  #convert back serialise data to original datatype
  print(loaded_index)

{'loving': {'file391.txt', 'file1.txt', 'file723.txt', 'file254.txt'}, 'vintage': {'file907.txt', 'file827.txt', 'file674.txt', 'file895.txt', 'file422.txt', 'file1.txt', 'file847.txt', 'file197.txt', 'file494.txt', 'file936.txt', 'file439.txt', 'file638.txt', 'file725.txt', 'file737.txt', 'file278.txt', 'file597.txt', 'file51.txt', 'file150.txt'}, 'springs': {'file806.txt', 'file937.txt', 'file1.txt', 'file272.txt', 'file469.txt'}, 'strat': {'file565.txt', 'file579.txt', 'file725.txt', 'file691.txt', 'file440.txt', 'file345.txt', 'file559.txt', 'file838.txt', 'file652.txt', 'file253.txt', 'file353.txt', 'file245.txt', 'file457.txt', 'file611.txt', 'file163.txt', 'file1.txt', 'file197.txt', 'file993.txt', 'file25.txt', 'file469.txt', 'file940.txt', 'file801.txt', 'file853.txt', 'file149.txt', 'file90.txt', 'file529.txt', 'file455.txt', 'file626.txt', 'file978.txt', 'file241.txt', 'file396.txt', 'file400.txt', 'file519.txt', 'file422.txt', 'file650.txt', 'file380.txt'}, 'good': {'file29

In [12]:
for file in os.listdir("Part2_user_input"):
  input_file_path=os.path.join("Part2_user_input",file)
  output_file_path=os.path.join("Part2_processed_input",file)
    
  with open(input_file_path,'r',encoding='utf-8') as input_file:
   #Reading only the first line
    text=input_file.readline().strip()
    text=lowercase_data(text)
    tokens=tokenize_Data(text)
    tokens=remove_stopwords(tokens)
    tokens=remove_punctuations(tokens)
    tokens=remove_blank_space(tokens)
    oper=input_file.readline().strip()

    processed_data=' '.join(tokens)
    with open(output_file_path,'w',encoding='utf-8') as output_file:
        output_file.write(processed_data+'\n')
        output_file.write(oper)
print("complete")

complete


In [13]:
def perform(op,set1,set2):
  if(op=='AND'):
    return set1&set2
  if(op=='OR'):
    return set1|set2
  elif (op=='AND NOT'):
    return set1-set2
  elif (op=='OR NOT'):
    return set1.symmetric_difference(set2)
  else:
    return set()

In [14]:
def output_generate(Part2_processed_input):
    query_no = 0
    for file in os.listdir(Part2_processed_input):
        input_path = os.path.join(Part2_processed_input, file)

        with open(input_path, 'r', encoding='utf-8') as input_file:
            query_no += 1
            query_words = input_file.readline().strip().split()
            operations = input_file.readline().strip().split(',')

            combine_query =""
            for i in range(len(query_words)-1):
                combine_query+=f"{query_words[i]} {operations[i]} "
            combine_query+=query_words[-1]

            result=None
            op_index = 0

            for i in range(len(query_words) - 1):
                if result is None:
                    result = loaded_index.get(query_words[i], set())
                else:
                    next_set = loaded_index.get(query_words[i+1], set())
                    result = perform(operations[op_index], result, next_set)
                    op_index += 1 if op_index < len(operations) - 1 else 0

            output_str = f"Query {query_no}: {combine_query}\n"
            output_str += f"Number of documents retrieved for query {query_no}: {len(result)}\n"
            output_str += f"Names of the documents retrieved for query {query_no}: "
            if result:
                output_str += ', '.join(result)
            else:
                output_str += "No documents found"

            print(output_str)

In [15]:
output_generate("Part2_processed_input")

Query 1: car OR bag AND NOT canister
Number of documents retrieved for query 1: 6
Names of the documents retrieved for query 1: file542.txt, file174.txt, file746.txt, file264.txt, file166.txt, file886.txt
Query 2: coffee AND brewing  OR NOT techniques  OR cookbook
Number of documents retrieved for query 2: 0
Names of the documents retrieved for query 2: No documents found
